In [12]:
import pandas as pd
import numpy as np
import os
from pathlib import Path

# This ensures it saves to Desktop/newData regardless of OS
OUTPUT_DIR = Path.home() / "Desktop" / "newData"

# Loading tables
student_info = pd.read_csv("studentInfo.csv")
student_vle = pd.read_csv("studentVle.csv")
student_reg = pd.read_csv("studentRegistration.csv")
assessments = pd.read_csv("assessments.csv")
student_assessment = pd.read_csv("studentAssessment.csv")

# Creating unique key for each student
# pandas applies this operation to every row
student_info['student_key'] = (
    student_info['code_module'] + '_' +
    student_info['code_presentation'] + '_' +
    student_info['id_student'].astype(str)
)

# Stratifying split by final_result
np.random.seed(42) # For reproducibility
train_keys, val_keys, test_keys = set(), set(), set()

for result in student_info['final_result'].unique():
    subset = student_info[student_info['final_result'] == result]
    keys = subset['student_key'].unique()
    np.random.shuffle(keys)

    # Total number of students with this final result
    total_students = len(keys)

    num_train = int(0.6 * total_students)
    num_val = int(0.2 * total_students)
    # The rest will go to the test set

    # Adding multiple elements with .update()
    train_keys.update(keys[0:n_train]) # 60%
    val_keys.update(keys[n_train:n_train + n_val]) # 20%
    test_keys.update(keys[n_train + n_val:len(keys)]) # the remaining 20%

# Key columns
key_cols = ['code_module', 'code_presentation', 'id_student']

# Function to filter a DataFrame based on selected student keys
def filter_by_keys(df, key_columns, selected_keys):
    # 1. Creating a new column called 'student_key' in the given DataFrame
    df['student_key'] = (
        df[key_columns[0]] + '_' +                # e.g., 'F2'
        df[key_columns[1]] + '_' +                # e.g., '2014J'
        df[key_columns[2]].astype(str)            # e.g., '654321'
    )
    # student_key becomes something like "F2_2014J_654321"

    # 2. Keeping only the rows where this student_key is in the selected_keys set
    df_filtered = df[df['student_key'].isin(selected_keys)]

    # 3. Dropping the student_key column to return the DataFrame to its original shape
    return df_filtered.drop(columns=['student_key'])

# .copy() creates a separate copy
train_info = filter_by_keys(student_info.copy(), key_cols, train_keys)
val_info = filter_by_keys(student_info.copy(), key_cols, val_keys)
test_info = filter_by_keys(student_info.copy(), key_cols, test_keys)

train_vle = filter_by_keys(student_vle.copy(), key_cols, train_keys)
val_vle = filter_by_keys(student_vle.copy(), key_cols, val_keys)
test_vle = filter_by_keys(student_vle.copy(), key_cols, test_keys)

train_reg = filter_by_keys(student_reg.copy(), key_cols, train_keys)
val_reg = filter_by_keys(student_reg.copy(), key_cols, val_keys)
test_reg = filter_by_keys(student_reg.copy(), key_cols, test_keys)

# Adding module and presentation to student_assessment
student_assessment = pd.merge(
    student_assessment,
    assessments[['id_assessment', 'code_module', 'code_presentation']],
    on='id_assessment',
    how='left'
)

train_assessment = filter_by_keys(student_assessment.copy(), key_cols, train_keys)
val_assessment = filter_by_keys(student_assessment.copy(), key_cols, val_keys)
test_assessment = filter_by_keys(student_assessment.copy(), key_cols, test_keys)


def save_splitted_data(train_data, val_data, test_data, table):
    # Creating a directory at the given path.
    # exist_ok=True means that if the folder already exists, don’t raise an error, just continue.
    os.makedirs(os.path.join(OUTPUT_DIR, "train"), exist_ok=True)
    os.makedirs(os.path.join(OUTPUT_DIR, "val"), exist_ok=True)
    os.makedirs(os.path.join(OUTPUT_DIR, "test"), exist_ok=True)

    train_data.to_csv(os.path.join(OUTPUT_DIR, "train", f"{table}.csv"), index=False)
    val_data.to_csv(os.path.join(OUTPUT_DIR, "val", f"{table}.csv"), index=False)
    test_data.to_csv(os.path.join(OUTPUT_DIR, "test", f"{table}.csv"), index=False)

save_splitted_data(train_info, val_info, test_info, 'student_info')
save_splitted_data(train_vle, val_vle, test_vle, 'student_vle')
save_splitted_data(train_reg, val_reg, test_reg, 'student_registration')
save_splitted_data(train_assessment, val_assessment, test_assessment, 'student_assessment')

print("Final result (student_info) distribution in each set:\n")

for label in student_info['final_result'].unique():
    train_count = (train_info['final_result'] == label).sum()
    val_count = (val_info['final_result'] == label).sum()
    test_count = (test_info['final_result'] == label).sum()
    total = train_count + val_count + test_count

    print(f"{label}:")
    print(f"Train: {train_count} ({train_count/total:.2%})")
    print(f"Val:   {val_count} ({val_count/total:.2%})")
    print(f"Test:  {test_count} ({test_count/total:.2%})\n")

print("Distribution result by tables:\n")

def print_counts(name, train_df, val_df, test_df):
    print(f"{name}:")
    print(f"Train: {len(train_df)}")
    print(f"Val:   {len(val_df)}")
    print(f"Test:  {len(test_df)}\n")

print_counts("Student Info", train_info, val_info, test_info)
print_counts("Student VLE", train_vle, val_vle, test_vle)
print_counts("Student Registration", train_reg, val_reg, test_reg)
print_counts("Student Assessment", train_assessment, val_assessment, test_assessment)

Final result (student_info) distribution in each set:

Pass:
Train: 1814 (14.68%)
Val:   604 (4.89%)
Test:  9943 (80.44%)

Withdrawn:
Train: 1814 (17.86%)
Val:   604 (5.95%)
Test:  7738 (76.19%)

Fail:
Train: 1814 (25.72%)
Val:   604 (8.56%)
Test:  4634 (65.71%)

Distinction:
Train: 1814 (59.99%)
Val:   604 (19.97%)
Test:  606 (20.04%)

Distribution result by tables:

Student Info:
Train: 7256
Val:   2416
Test:  22921

Student VLE:
Train: 2089547
Val:   719804
Test:  5649969

Student Registration:
Train: 7256
Val:   2416
Test:  22921

Student Assessment:
Train: 41177
Val:   13755
Test:  118980

